In [3]:
from dotenv import load_dotenv

load_dotenv()
import os
import discord
import asyncio
from tqdm.auto import tqdm
from IPython.display import display, HTML
from discord.utils import get
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
from collections import Counter
from datetime import datetime
from dateutil import tz


In [36]:
class DiscordClient(discord.Client):
    def __init__(self):
        intents = discord.Intents.default()
        intents.members = True
        super().__init__(intents=intents)

    async def cbt(self):  # call back task
        while not self.is_ready():
            await asyncio.sleep(0.2)


client = DiscordClient()
TOKEN = os.getenv("REEBOT_TOKEN")
IS_BOT = 1  # <= 1.7.3
await asyncio.wait(
    [
        asyncio.create_task(client.start(TOKEN, bot=IS_BOT)),
        asyncio.create_task(client.cbt()),
    ],
    return_when=asyncio.FIRST_COMPLETED,
)
print(f"Logged in as {client.user}")


Logged in as Reon Mode#3954


### get channels

In [37]:
NAME = "poi"

g_ls = [g for g in client.guilds if NAME in g.name.lower()]
guild = g_ls[0]
print(guild)
print(guild.members.__len__())
chnls = guild.text_channels
chnls


Poi's Bakery
295


[<TextChannel id=551585078816276482 name='🌟-welcome-🌸' position=0 nsfw=False news=False category_id=1084748648073658438>,
 <TextChannel id=761032628660338699 name='roles-and-rules' position=1 nsfw=False news=False category_id=1084748648073658438>,
 <TextChannel id=1084748314177712138 name='announcements' position=2 nsfw=False news=False category_id=1084748648073658438>,
 <TextChannel id=953983163393278002 name='log' position=3 nsfw=False news=False category_id=1037709533327405096>,
 <TextChannel id=1017847544551837786 name='divine-tier-only' position=4 nsfw=False news=False category_id=1037709533327405096>,
 <TextChannel id=842645369128091650 name='moderator-only' position=5 nsfw=False news=False category_id=1037709533327405096>,
 <TextChannel id=950982603220082718 name='bork-bork-general' position=6 nsfw=False news=False category_id=950984036778663936>,
 <TextChannel id=950106361860075641 name='🥛-milk-sauce' position=7 nsfw=True news=False category_id=950984036778663936>,
 <TextChanne

### Channel analyze

In [ ]:
chnl = client.get_channel(966672080647098458)
async for msg in chnl.history(limit=40):
    print(f'\033[1m{msg.author.name}:\033[0m {msg.content}')


In [ ]:
msg_list = []

In [ ]:
chnl = client.get_channel(950106361860075641)
LIMIT = 85_000
try:
    before = msg_list[-1].created_at
except IndexError:
    before = None
async for msg in tqdm(
    chnl.history(limit=LIMIT, before=before), initial=msg_list.__len__(), total=LIMIT
):
    msg_list.append(msg)


In [ ]:
msg_list[0].content.endswith(">")

In [ ]:
senders = [msg.author.name + "#" + msg.author.discriminator for msg in msg_list]
Counter(senders).most_common()[:20]


In [ ]:
senders = [
    msg.author.name + "#" + msg.author.discriminator
    for msg in msg_list
    if not (msg.content.startswith("<") and msg.content.endswith(">"))
]
Counter(senders).most_common()[:20]


### Messages analyze

In [ ]:
TZ = tz.gettz("Asia / Hanoi")
dt = datetime.strptime("21:01, 12/09/2020", "%H:%M, %d/%m/%Y")
dt.astimezone(TZ)
dt

In [ ]:
async def find_msg():
    for chnl in tqdm(chnls):
        try:
            async for msg in chnl.history(limit=100, before=dt):
                if "đu" in msg.content:
                    return msg
        except discord.Forbidden:
            print("Skipping:", chnl.name, "(Forbidden)")
            continue


await find_msg()


### Role analyze

In [23]:
guild.roles

[<Role id=551585078812082228 name='@everyone'>,
 <Role id=1074491013395124346 name='Maid'>,
 <Role id=1011625385974313063 name='Reon Bot'>,
 <Role id=989202305897336905 name='Lunachii'>,
 <Role id=975649092380278817 name='PartyBeast'>,
 <Role id=974890796727799821 name='Translator'>,
 <Role id=973985105737564243 name='Sun v5'>,
 <Role id=961297404588617839 name='Sun v4'>,
 <Role id=958373424487612480 name='Sun v3'>,
 <Role id=958338504247640107 name='Sun v2'>,
 <Role id=950015819809714189 name='Sun'>,
 <Role id=958279308772794411 name='NotSoBot'>,
 <Role id=952948284140445789 name='Yggdrasil'>,
 <Role id=946414355568803953 name='Karuta'>,
 <Role id=775676907390763018 name='Erin'>,
 <Role id=953218927788441603 name='carl-bot'>,
 <Role id=1015571461336997931 name='-- integrated --'>,
 <Role id=874286874804506634 name='Assignment Ping'>,
 <Role id=778970479385772032 name='Filthy Mine(craft?)rs'>,
 <Role id=763258253357350933 name='operationrigdhi'>,
 <Role id=761442228136312843 name='movi

In [ ]:
target_roles = [
    r
    for r in guild.roles
    if (
        any(s in r.name.lower() for s in ["bread", "maid"])
        and not any(s in r.name.lower() for s in ["supplier"])
    )
]
target_roles


In [ ]:
to_get = [679924727011344481, 551715823874473984, 551586677089501194]
level_roles = [get(guild.roles, id=x) for x in to_get]
level_roles


In [ ]:
display_mems = []
set_1_mems = set()
set_2_mems = set()
set_3_mems = set()
for mem in tqdm(guild.members):
    has_target_roles = any(r in mem.roles for r in target_roles)
    has_level_roles = any(r in mem.roles for r in level_roles)
    if has_target_roles:
        set_1_mems.add(mem)
    if has_level_roles:
        set_2_mems.add(mem)
    if level_roles[0] in mem.roles:
        set_3_mems.add(mem)
    if mem.roles.__len__() == 1:
        display_mems.append(mem)

print(display_mems.__len__(), "members")
for mem in display_mems:
    display(
        HTML(
            f"""
        <img src={mem.avatar_url} height=40>
        <b>{mem.name}</b>
    """
        )
    )


In [ ]:
out = "".join(
    [
        f"""
            <img src={mem.avatar_url} height=40>
            <b>{mem.name}</b><br>
        """
        for mem in display_mems
    ]
)
with open("../../Output/mid_tier_non_furry.html", "w") as f:
    f.write(out)


In [ ]:
venn3(
    [set_1_mems, set_2_mems, set_3_mems],
    ("Bread or Bot roles", "Tier roles", "Supreme tier members"),
)
plt.show()


### Member analyse

#### Filter members

In [33]:
roles1 = [r for r in guild.roles if "div" in r.name.lower()]
white_role = roles1[0]


In [39]:
set1 = set(client.get_channel(552861577204989973).members)  # in announceement old
set2 = set(client.get_channel(950982603220082718).members)  # in bork bork
set3 = set([m for m in guild.members if white_role in m.roles])  # has divine tier
mem_ls = set1 - set2 - set3
[(m.name, *[r.name for r in m.roles][1:]) for m in mem_ls]


[('sabi', 'Yametars', 'Love Birds'),
 ('CrystaL FC', '🎀'),
 ('Walu', 'OG', 'Artist', '🎀'),
 ('limitless', 'Artist', 'Supreme Tier', '🎀'),
 ('Igga',
  'Assignment Ping',
  'operationrigdhi',
  'OG',
  'Yametars',
  'Ironicus Caesar',
  'JAILTIME',
  'Artist',
  'Mid Tier',
  'Supreme Tier',
  'Greedy & Unfaithful',
  'Submissive and Breedable',
  'Dominant',
  'Love Birds'),
 ('Sunmusicbot', 'Sun', '🎀')]

## Write

#### Mass kick

In [35]:
for m in mem_ls:
    print(m)
    try:
        await m.kick(reason="Inactive")
    except Exception as e:
        print(e)

᲼᲼#2012
WhoTheLimon#0583
Musafir#0117
ganjababa#6238
𝐰𝐚𝐯𝐞𝐬.#8236
wakiflores#4744
Secretariat#3724
Drewwy#7599
Ripper66647#1587
Tsarbomb4#3769
RifahTasfia#2956
aoi#5190
M1rr0r#4493
KINGMASTER#4519
XPRO49 GAMING#2607
Tasiya#5937
Arcturus#5365
CheemsBorger#7626
Apple69#8606
ggemernoob#1138
sahed#7209
Z Factor#3751
Mahtab Shimon#8199
aalabir0511#9681
Weird Godfather#4606
SH4F1N#1091
Palbro#9054
chromaticv1#6232
farhan1151#2734
Edward Philips#0052
Rakin#5834
MubTheNub#6340
SalmanSR4k#6911
Raisul#2672
Electrolane#2135
BadamiPasa69#6909
Astrop#1151
Angered Doggo#6117
Erasha Elaba#4190
Ro-#7152
SsMahin#9661
Rabbi Khan#4417
limitless#0450
403 Forbidden (error code: 50013): Missing Permissions
Natalie#6206
mocona#0587
samia islam#4510
kingfish#2387
sabi#2629
403 Forbidden (error code: 50013): Missing Permissions
ᴉʇɹɐɔ#7670
THESILENTWALKR🇧🇩#0708
AZAZEL#9808
NajuBABA#3092
Passionate Book Reader#0401
Ironicus#6491
couch Potato#6159
Cute_Virus#6615
Albab#2795
Hiraeth#7971
Navid#8073
A47xMrBurger#188